In [1]:
# Setting up required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, SoupStrainer

In [4]:
# Connect to website to be scraped, and get all html
url1 = "https://results.virginmoneylondonmarathon.com/2021/?page="
url2 = "&event=ALL&num_results=1000&pid=search&pidp=results_nav&search%5Bsex%5D="
url3 = "&search%5Bage_class%5D=%25&search%5Bnation%5D=%25&search_sort=name"

# Get results for men, 3 pages of results <-There is no search option for other gender/sex
mens_results = pd.DataFrame()
for i in range(3):  # max 25
    sex = "M"

    site = requests.get(url1 + str(i) + url2 + sex + url3).text
    # Soup strainer restricts content to speed up soup
    strainer = SoupStrainer(class_="section-main")
    soup = BeautifulSoup(site, "lxml", parse_only=strainer)
    # fields = soup.find(class_='section-main')

    # Loop through each row and column to create a list of cells
    my_table = []
    for row in soup.find_all(class_="list-group-item"):
        row_data = []
        for cell in row.find_all(class_="list-field"):
            row_data.append(cell.text)
        # If the row isn't empty, then create a dict of the row to create dataframe from
        if row_data:
            data_item = {
                "Place (Overall)": row_data[0],
                "Place (Gender)": row_data[1],
                "Place (Category)": row_data[2],
                "Name": row_data[3],
                "Sex": sex,
                "Club": row_data[4],
                "Running Number": row_data[5],
                "Category": row_data[6],
                "Finish": row_data[9],
                "Year": 2021,
            }
            my_table.append(data_item)
    df = pd.DataFrame(my_table).iloc[1:]  # Strip table header

    mens_results = mens_results.append(df)

SyntaxError: invalid syntax (<string>, line 1)

In [12]:
# Get results for women
womens_results = pd.DataFrame()
for i in range(3):  # 17
    sex = "F"

    site = requests.get(url1 + str(i) + url2 + sex + url3).text
    # Soup strainer restricts content to speed up soup
    strainer = SoupStrainer(class_="section-main")
    soup = BeautifulSoup(site, "lxml", parse_only=strainer)
    # fields = soup.find(class_='section-main')

    # Loop through each row and column to create a list of cells
    my_table = []
    for row in soup.find_all(class_="list-group-item"):
        row_data = []
        for cell in row.find_all(class_="list-field"):
            row_data.append(cell.text)
        # If the row isn't empty, then create a dict of the row to create dataframe from
        if row_data:
            data_item = {
                "Place (Overall)": row_data[0],
                "Place (Gender)": row_data[1],
                "Place (Category)": row_data[2],
                "Name": row_data[3],
                "Sex": sex,
                "Club": row_data[4],
                "Running Number": row_data[5],
                "Category": row_data[6],
                "Finish": row_data[9],
                "Year": 2021,
            }
            my_table.append(data_item)
    df = pd.DataFrame(my_table).iloc[1:]  # Strip table header

    womens_results = womens_results.append(df)

In [14]:
# Concatenate results
results = pd.concat([mens_results, womens_results])

In [15]:
results

,Place (Overall),Place (Gender),Place (Category),Name,Sex,Club,Running Number,Category,Finish,Year
1,12547,9390,4345,"A Smith, Matthew (GBR)",M,Club–,Runner Number40546,Category18-39,Finish03:59:33,2021
2,34146,20833,3132,"Aalders, Steven (GBR)",M,Club–,Runner Number23235,Category45-49,Finish06:22:20,2021
3,–,–,–,"Abarca, Octavio (GBR)",M,Club–,Runner Number52134,Category50-54,Finish–,2021
4,–,–,–,"Abbas, Louis (GBR)",M,ClubMornington Chasers,Runner Number24700,Category40-44,Finish–,2021
5,33974,20770,1381,"Abbasi, Naveed Shah (GBR)",M,Club–,Runner Number7204,Category55-59,Finish06:18:08,2021
...,...,...,...,...,...,...,...,...,...,...
996,8806,6880,1055,"Barnett, Simeon (GBR)",F,Club–,Runner Number29796,Category45-49,Finish03:44:51,2021
997,28997,18705,234,"barnett, tony (GBR)",F,Club–,Runner Number46334,Category65-69,Finish05:21:38,2021
998,–,–,–,"Barnfather, Kris",F,Club–,Runner Number11530,Category40-44,Finish–,2021
999,4888,694,359,"Barnish, Heather (GBR)",F,Club–,Runner Number24850,Category18-39,Finish03:23:52,2021


In [16]:
# Some quick data cleaning
# Remove leftover titles
results["Club"] = results["Club"].str.replace("Club", "", regex=False)
results["Running Number"] = results["Running Number"].str.replace(
    "Running Number", "", regex=False
)
results["Category"] = results["Category"].str.replace("Category", "", regex=False)
results["Finish"] = results["Finish"].str.replace("Finish", "", regex=False)

# Extract country groups, like (USA), from Name group
results["Country"] = results["Name"].str.extract(r"(\([A-Z]{3,}\))")
# Remove brackets in country
results["Country"] = results["Country"].str.replace(r"\(|\)", "", regex=True)
# Remove country group from name column
results["Name"] = results["Name"].str.replace(r"(\([A-Z]{3}\))", "", regex=True)

# Split first/lastname into new columns
LastFirst = results["Name"].str.split(pat=",", n=1, expand=True)
results["FirstName"], results["LastName"] = LastFirst[1], LastFirst[0]
# Remove comma from Name column, so that this can be saved as a CSV ----- Must happen after splitting Name into two cols!!
results["Name"] = results["Name"].str.replace(r"(\,)", "", regex=True)
# Replace non-standard '–' with NaN for missing vals
results = results.replace("–", np.nan)
results = results.replace("DSQ", np.nan)
results = results.replace("", np.nan)

# Delete odd race number row - table description not actual data
results = results.loc[
    (results["Running Number"] != "RM9999") & (results["Running Number"] != "RF9999")
]

# Set data types
results = results.astype(
    {
        "Place (Overall)": "float64",
        "Place (Gender)": "float64",
        "Place (Category)": "float64",
        "Name": str,
        "Sex": str,
        "Club": str,
        "Running Number": str,
        "Category": "category",
        "Country": str,
        "FirstName": str,
        "LastName": str,
    }
)
# Due to an irritating bug with converting objects to Int64, needed to first convert to float and then to int
results = results.astype(
    {"Place (Overall)": "Int64", "Place (Gender)": "Int64", "Place (Category)": "Int64"}
)
results["Finish"] = pd.to_timedelta(results["Finish"])
results["Finish (Total Seconds)"] = results["Finish"].dt.total_seconds()

In [17]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 1 to 1000
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   Place (Overall)         5078 non-null   Int64          
 1   Place (Gender)          5078 non-null   Int64          
 2   Place (Category)        5078 non-null   Int64          
 3   Name                    6000 non-null   object         
 4   Sex                     6000 non-null   object         
 5   Club                    6000 non-null   object         
 6   Running Number          6000 non-null   object         
 7   Category                5998 non-null   category       
 8   Finish                  5078 non-null   timedelta64[ns]
 9   Year                    6000 non-null   int64          
 10  Country                 6000 non-null   object         
 11  FirstName               6000 non-null   object         
 12  LastName                6000 non-n

In [19]:
# And quickly save them in a csv
results.to_csv(
    r"./London_2021.csv",
    index=False,
    header=True,
)